In [1]:
import tensorflow as tf
import gym
from IPython.display import clear_output
import time
import numpy as np

In [2]:
env=gym.make('Alien-v0')
obs=env.reset()
print(env.action_space)

Discrete(18)


In [3]:
from PIL import Image
resize_x=64
resize_y=64
def preprocessing(obs):
    buf=[]
    for i in range(len(obs)):
        xbuf=[]
        for j in range(len(obs[i])):
            gray_pixel=obs[i][j][0]*0.299+obs[i][j][1]+0.587+obs[i][j][2]*0.114
            xbuf.append(gray_pixel)
        buf.append(xbuf)
    #return np.array(buf)
    nparr=np.array(buf)
    image=Image.fromarray(nparr).resize((resize_x,resize_y))
    return np.array(image)#image
          
def inqueue(queue,obs,que_size):
    queue.append(obs)
    if len(queue)>4:
        queue.pop(0)
    return queue
#test code

In [4]:

score=0
step=0
rewards=[]
frame_queue=[]
queue_size=4


#Random Agent
score=0
step=0
rewards=[]
frame_queue=[]
queue_size=4

while True:
    action=env.action_space.sample()
    obs,reward,done,info=env.step(action)
    #env.render()
    frame_queue=inqueue(frame_queue,obs,queue_size)
    
    score+=reward
    rewards.append(reward)
    step+=1
    if done:
        break

print(f'score>{score}\tstep>{step}')

import matplotlib.pyplot as plt
for i in range(len(frame_queue)):
    
    plt.subplot(len(frame_queue)+1,1,i+1)
    plt.imshow(np.asarray(frame_queue[0]),cmap='gray')
plt.show()

In [8]:
def xyz2zxy(xyzArray):
    zxyArray=np.zeros((len(xyzArray[0]),len(xyzArray[0][0]),len(xyzArray)))
    for i in range(len(xyzArray)):
        for j in range(len(xyzArray[i])):
            for k in range(len(xyzArray[i][j])):
                zxyArray[j][k][i]=xyzArray[i][j][k]
    return zxyArray
def buildmodel():
    model=tf.keras.Sequential([
            tf.keras.layers.Conv2D(input_shape=(resize_x,resize_y,queue_size,), kernel_size=(8,8), filters=16),
            tf.keras.layers.MaxPool2D(strides=(2,2)),
            tf.keras.layers.Conv2D(kernel_size=(4,4),filters=32),
            tf.keras.layers.MaxPool2D(strides=(2,2)),
            tf.keras.layers.Conv2D(kernel_size=(4,4),filters=32),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(units=256, activation='relu'),
            tf.keras.layers.Dense(units=18, activation='softmax')
        ])
    model.compile(optimizer=tf.optimizers.Adam(),
                  loss='sparse_categorical_crossentropy', 
                  metrics=['accuracy'])
    return model
model=buildmodel()
target_model=buildmodel()
model.summary()
target_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 57, 57, 16)        4112      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 28, 28, 16)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 25, 25, 32)        8224      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 9, 9, 32)          16416     
_________________________________________________________________
flatten_2 (Flatten)          (None, 2592)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 256)              

In [9]:
gamma=0.9
batch_size=512
max_memory=batch_size*16
memory=[]

def append_sample(mem,state,action,reward,next_state,done):
    mem.append([state,action,reward,next_state,done])
    
def train_model(c_model,t_model):
    np.random.shuffle(memory)
    lenth=max_memory//batch_size
    for i in length:
        mini_batch=memory[k*batch_size:(k+1)*batch_size]
        states=np.zeros((batch_size,resize_x,resize_y,queue_size))
        next_states=np.zeros((batch_size,resize_x,resize_y,queue_size))
        actions,rewards,dones=[],[],[]
        
        
        for i in range(batch_size):
            states[i]=mini_batch[i][0]
            actions.append(mini_batch[i][1])
            rewards.append(mini_batch[i][2])
            next_states[i]=mini_batch[i][3]
            dones.append(mini_batch[i][4])
            
        target=c_model.predict(states)
        next_target=t_model.predict(next_states)
        
        for i in range(batch_size):
            if dones[i]:
                target[i][actions[i]]=rewards[i]
            else:
                target[i][actions[i]]=rewards[i]+gamma*np.amax(next_target[i])
        model.fit(states,target,batch_size=batch_size,epochs=2,verbose=0)

In [11]:
import random
import math 

def softmax(logits):
    exp_logits=np.exp(logits-np.max(logits))
    sum_exp_logits=np.sum(exp_logits)
    return exp_logits/sum_exp_logits
max_episodes=1001
epsilon=0.9
epsilon_min=0.1

scores=[]
steps=[]
iteration=0
train_count=0
print(env._max_episode_steps)
for i in range(max_episodes):
    #if i%100==1 and i!=0:
        #print(f'{i}\tscore mean>{np.mean(scores[-100:])}\tstep mean>{np.mean(steps[-100:])}\titeration>{iteration}\tepsilon>{epsilon}')
    print(f'{i}\tscore mean>{np.mean(scores[-100:])}\tstep mean>{np.mean(steps[-100:])}\titeration>{iteration}\tepsilon>{epsilon}')
    prev_obs=preprocessing(env.reset())
    score=0
    step=0
    
    
    while step<env._max_episode_steps/10:
        iteration+=1
        if random.random()<epsilon and len(frame_queue)<4:
            action=env.action_space.sample()
        else:
            x=prev_obs
            val=np.expand_dims(xyz2zxy(frame_queue),axis=0)
            logits=model.predict(val)
            prob=softmax(logits)
            action=np.argmax(prob)
        obs,reward,done,info=env.step([action])
        processing_obs=preprocessing(obs)
        frame_queue=inqueue(frame_queue,processing_obs,queue_size)
        score+=reward
        step+=1
        
        #custom reward
        append_sample(memory,prev_obs,action,reward,processing_obs,done)
        
        if len(memory)>=max_memory:
            train_model(model,target_model)
            memory=[]
            train_count+=1
            if train_count%4==0:
                target_model.set_weights(model.get_weights())
        prev_obs=processing_obs
        if epsilon>0.01 and iteration%2500==0:
            epsilon=epsilon/1.005
        if done:
            break
    scores.append(score)
    steps.append(step)
    print(f'{i}\tScore>{score}\tStep>{step}\tMax_tile>{np.max(obs)}\tmemory_len>{len(memory)}')
        

10000
0	score mean>nan	step mean>nan	iteration>0	epsilon>0.9
0	Score>0.0	Step>547	Max_tile>252	memory_len>778
1	score mean>0.0	step mean>547.0	iteration>547	epsilon>0.9


KeyboardInterrupt: 

import matplotlib.pyplot as plt
plt.figure(figsize=(10,10))

pix_size=10
max_len=int(len(obs)/pix_size*len(obs[i])/pix_size)
for i in range(int(len(obs)/pix_size)):
    for j in range(int(len(obs[i])/pix_size)):
        idx=i*21+j+1
        if idx > max_len:
            break
        buf=[]
        for k in range(pix_size):
            xbuf=[]
            for l in range(pix_size):
                xbuf.append(obs[i*10+k][j*10+l])
            buf.append(xbuf)
        
        plt.subplot(21,16,idx,label=np.mean(buf))
        plt.imshow(buf)
    
plt.show()

In [ ]:
import matplotlib.pyplot as plt
N=10
rolling_mean=[np.mean(scores[x:x+N]) for x in range(len(scores)-N+1)]
plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
plt.scatter(range(len(scores)),scores,marker='.')
plt.subplot(1,2,2)
plt.plot(rolling_mean)
plt.show()